In [48]:
#Tech Task: Task 2
import pandas as pd
import numpy as np
import requests
import sqlite3
from bs4 import BeautifulSoup
from time import sleep
import re


In [3]:
#Check the maximum number of crashes across all the years - max year is 1972 with 105 crashes
crashNum = {}

for i in range(1920, 2018):
    crashNum[i] = 0
    maxCrashNum = 105
    url = "http://www.planecrashinfo.com/{}/{}-{}.htm".format(i, i, maxCrashNum)
        
    try:
        r = requests.get(url)
        if r.ok:
            print("{} has more than maxCrashNum".format(i))
        else:
            print(".", end="")
            
    #Insert delay so to avoid connection being refused (Max retries exceeded with URL)
    except requests.exceptions.ConnectionError:
            
            print("Connection refused")
            sleep(5)
        

....................................................1972 has more than maxCrashNum
.............................................

In [4]:
#Get data from crash website and put in crashPages list

crashPages = list()

for year in range(1920, 2018):
    for i in range(1,110):
        url = "http://www.planecrashinfo.com/{}/{}-{}.htm".format(year, year, i)
        try:
            r = requests.get(url)
            if not r.ok:
                break
            else:
                crashPages.append(r)
                print(".", end="")
    
        #Insert delay so to avoid connection being refused (Max retries exceeded with URL)
        except requests.exceptions.ConnectionError:
            
            print("Connection refused")
            sleep(5)
    

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [5]:
crashPages[1].ok

True

In [77]:
#Parse data in crashPages list

recordList = list()
del recordList[:] #clear list

for page in crashPages:
    soup = BeautifulSoup(page.text, 'html.parser')

    table = soup.find('table')
    rows = table.find_all('tr')
    rows = rows[1:]
    
    dataDict = {}

    for row in rows:
        td_items = row.find_all('td')
        td_label = td_items[0].text.strip()
        td_data = td_items[1].text.strip()
        dataDict[td_label] = td_data
    
    #Clean data
    try:
        aboardNum = dataDict['Aboard:'].split(" ")[0] #Just take the first number, don't worry about the passenger/crew breakup
        dataDict['Aboard:'] = int(aboardNum)
        fatalityNum = dataDict['Fatalities:'].split(" ")[0] #Just take the first number, don't worry about the passenger/crew breakup
#         print("{}, ".format(fatalityNum), end="")
        dataDict['Fatalities:'] = int(fatalityNum) 
    except:
#         print('x', end="") #corrupted data
        continue
    
    recordList.append(dataDict)

In [72]:
#Insert into pandas dataframe (crashFrame)
crashFrame.drop(crashFrame.index, inplace=True) #clear dataframe
crashFrame = pd.DataFrame(recordList)

In [73]:
crashFrame['Fatalities:'].unique()

array([  1,   5,  14,  30,  21,  19,  20,  22,  27,  23,  24,  18,   0,
         2,   4,   3,  46,  34,   7,  17,   6,  52,   8,   9,  16,  10,
        48,  15,  73,  12,  11,  50,  13,  35,  25,  29,  36,  26,  40,
        31,  28,  59,  45,  44,  39,  32,  37,  38,  53,  43,  41,  33,
        55,  80,  58,  56,  70,  87, 129,  42,  47,  66,  74, 128,  62,
        67,  79,  61,  49,  99,  64,  51,  65,  60,  68,  54,  63,  72,
        78,  83,  77,  76,  95, 111, 107, 130, 112,  81,  84,  94,  86,
        97, 101, 118,  85,  57, 121, 117, 133, 124,  98,  82, 126,  88,
       123, 155, 100, 120,  90,  92, 102, 109,  75,  93, 163,  69,  91,
       104, 115, 122, 156, 108, 174, 103, 176, 110,  89, 106, 346, 191,
       113, 188, 154,  96, 583, 131, 213, 137, 183, 271, 178, 257, 146,
       166, 301, 180, 132, 145, 119, 269, 181, 148, 329, 200, 134, 520,
       256, 167, 159, 143, 290, 259, 144, 171, 127, 223, 261, 158, 141,
       157, 116, 264, 160, 189, 230, 349, 228, 234, 196, 229, 21

In [79]:
# Total fatalities between period 1920-2016 period

total_fatalities = crashFrame['Fatalities:'].sum()
print("Total fatalities from 1920-2016: {}".format(total_fatalities))

Total fatalities from 1920-2016: 114575


In [83]:
# Top 3 airlines with the highest rate of incidents

airline_incident_frame = crashFrame
print("Total fatalities from 1920-2016: {}".format(total_fatalities))


# Year with the highest incidents
# crashFrame.replace('?', np.nan, inplace=True)
crashFrame.head(1000)

,AC Type:,Aboard:,Date:,Fatalities:,Flight #:,Ground:,Location:,Operator:,Registration:,Route:,Summary:,Time:,cn / ln:
0,Wright Flyer III,2,"September 17, 1908",1,?,0,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,"During a demonstration flight, a U.S. Army fly...",17:18,1
1,Wright Byplane,1,"September 07, 1909",1,?,0,"Juvisy-sur-Orge, France",?,SC1,Air show,Eugene Lefebvre was the first pilot to ever be...,?,?
2,Dirigible,5,"July 12, 1912",5,?,0,"Atlantic City, New Jersey",Military - U.S. Navy,?,Test flight,First U.S. dirigible Akron exploded just offsh...,06:30,?
3,Curtiss seaplane,1,"August 06, 1913",1,?,0,"Victoria, British Columbia, Canada",Private,?,?,The first fatal airplane accident in Canada oc...,?,?
4,Zeppelin L-1 (airship),20,"September 09, 1913",14,?,0,Over the North Sea,Military - German Navy,?,?,The airship flew into a thunderstorm and encou...,c 18:30,?
5,Zeppelin L-2 (airship),30,"October 17, 1913",30,?,0,"Near Johannisthal, Germany",Military - German Navy,?,?,Hydrogen gas which was being vented was sucked...,c 10:30,?
6,Zeppelin L-8 (airship),41,"March 05, 1915",21,?,0,"Tienen, Belgium",Military - German Navy,?,?,Crashed into trees while attempting to land af...,c 01:00,?
7,Zeppelin L-10 (airship),19,"September 03, 1915",19,?,0,"Off Cuxhaven, Germany",Military - German Navy,?,?,"Exploded and burned near Neuwerk Island, when...",c 15:20,?
8,Schutte-Lanz S-L-10 (airship),20,"July 28, 1916",20,?,0,"Near Jambol, Bulgeria",Military - German Army,?,?,"Crashed near the Black Sea, cause unknown.",?,?
9,Zeppelin L-32 (airship),22,"September 24, 1916",22,?,0,"Billericay, England",Military - German Navy,?,?,Shot down by British aircraft crashing in flames.,c 01:00,?


In [16]:
# print(crashFrame.columns.tolist())
crashFrame.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5772 entries, 0 to 5771
Data columns (total 13 columns):
AC
        Type:    5748 non-null object
Aboard:             5772 non-null object
Date:               5772 non-null object
Fatalities:         5772 non-null object
Flight #:           1337 non-null object
Ground:             5721 non-null object
Location:           5766 non-null object
Operator:           5751 non-null object
Registration:       5418 non-null object
Route:              4272 non-null object
Summary:            5540 non-null object
Time:               3654 non-null object
cn / ln:            4562 non-null object
dtypes: object(13)
memory usage: 586.3+ KB


In [20]:
# Total fatalities between period 1920-2016 period
# Top 3 airlines with the highest rate of incidents
# Year with the highest incidents

# crashFrame['Fatalities:'].replace(np.nan, 0, inplace=True)
totalFatalities = crashFrame['Fatalities:'].astype(int).sum()
print("Total fatalities in 1920-2016 period: {}".format(totalFatalities))


ValueError: invalid literal for int() with base 10: '? \xa0 (passengers:?\xa0 crew:?)'

In [70]:
SQLITE_FILE = "crashData.db"
conn = sqlite3.connect(SQLITE_FILE) #opens sqlite and database file
myCursor = conn.cursor() #provides connection to database
crashFrame.to_sql("crashTable", conn, if_exists="replace")


/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:1534: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)
